In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

In [2]:
cap = cv2.VideoCapture('sample.mp4') #reading the video
#frame_width = int( cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#frame_height =int( cap.get( cv2.CAP_PROP_FRAME_HEIGHT))
#fourcc = cv2.VideoWriter_fourcc('X','V','I','D')
#out = cv2.VideoWriter("output.avi", fourcc, 5.0, (1280,720))


#capturing 2 consecutive frames
ret, frame1 = cap.read()
ret, frame2 = cap.read()


while cap.isOpened():
    diff = cv2.absdiff(frame1, frame2) #getting the difference between both the frames, to find any moving object
    
    #getting the noise out of the detected object
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY) 
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations=3)
    
    #creating contours around the detected moving object
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    #Taking each contours and creating rectangle bounding box around the object
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        if cv2.contourArea(contour) < 500: #removing the small contours(this value can be adjusted according to desired object's size)
            continue         
        cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
    #cv2.drawContours(frame1, contours, -1, (0, 255, 0), 1)

    #image = cv2.resize(frame1, (1280,720))
    #out.write(image)
    cv2.imshow("feed", frame1)


    frame1 = frame2
    ret, frame2 = cap.read()

    if cv2.waitKey(40) == 27:
        break

cv2.destroyAllWindows()
cap.release()

##### 